In [26]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


### Datos de cada BPI

Defino las rutinas para capturar las métricas,

In [27]:
def direct_play(x):

    maxi,pos=0,[]

    for i in range(len(x)-1):

        e1,e2 = x[i],x[i+1]


        dt = (e2["eventSec"]-e1["eventSec"])

        if dt>2:

            if e1["eventName"]=="Pass" or e1["eventName"]=="Free Kick":

                dist_vertical = (e2['positions'][0]['x']- e1['positions'][0]['x'])/dt

                if dist_vertical>maxi:
                    maxi=dist_vertical
                


    return maxi


def counterattack(x):

    x0, xf = x[0]["positions"][0]["x"], x[-1]["positions"][0]["x"]

    if x0<40 and xf>60:

        dt = x[-1]["eventSec"]-x[0]["eventSec"]

        if dt>0:

            v = (xf-x0)/dt
            return v
        
    
    return 0


def T_maintenance(x):
    
    dt = x[-1]['eventSec'] - x[0]['eventSec']

    if dt>2:

        if np.mean([ e['positions'][0]['x'] for e in x])<40:
            return dt
        
        else:
            return 0.0
    
    else:
        return 0.0
    

def T_build_up(x):

    dt = x[-1]['eventSec'] - x[0]['eventSec']

    if dt>2:

        if np.mean([ e['positions'][0]['x'] for e in x])>60:
            return dt
        
        else:
            return 0.0
    
    else:
        return 0.0


def T_zona_media(x):

    dt = x[-1]['eventSec'] - x[0]['eventSec']

    if dt>2:

        xm = np.mean([ e['positions'][0]['x'] for e in x])

        if xm<=60 and xm>=40:
            return dt
        else:
            return 0.0
    
    else:
        return 0.0


def flow_rate(x):

    R,T=[],[]

    for i in range(len(x)-1):

        e1,e2 = x[i], x[i+1]
        dt = e2["eventSec"]-e1["eventSec"] 

        if dt>0:
            T.append(dt)
            R.append( e1['positions'][0]['x'] )

    if len(T) and len(R):

        dtmean = np.mean(T)

        if np.mean(R)>50:# and dtmean<5:
        
            return 1.0/dtmean
   

    return 0.0
    

def crossing(x):

    for i in range(len(x)):

        e1= x[i]

        if e1["subEventName"]=="Cross":
            return 1

    return 0


def pressure_point(x, Tant):

    if Tant>2:

        xp = x[0]['positions'][0]['x']

        return xp 
    else:
        return 0.0


def pressure_loss(x,Tant):

    if Tant>2 and x[0]['positions'][0]['x']>40:

        return Tant 
    
    else:
        return 0
    

def shots(x):

    if np.mean([ e['positions'][0]['x'] for e in x])>50:

        for e in x:
            if e["eventName"]=="Shot":
                return 1
    
    return 0


tomo los datos,

In [28]:
Data={
    "direct_play":[],
    "counterattack":[],
    "T_maintenance":[],
    "T_build_up":[],
    "T_zona_media":[],
    "flow_rate":[],
    "crossing":[],
    "pressure_point":[],
    "pressure_loss":[],
    "shots":[],

    "match":[],
    "team1":[],
    "liga":[]
}

ligas = ["England","France","Germany", "Italy", "Spain"]

for liga in ligas:

    print (liga)

    eventos = json.load(open("/home/chacoma/Lineas/Futbol/raw_data/pappalardo/events/events_%s.json"%liga,'r'))
    N=len(eventos)

    count=0
    posesion = []
    Tpos_ant=0
    team_ant =''
    eant={"eventName":""}

    for i in range(N-1):

        e1 = eventos[i]
        e2 = eventos[i+1]

        if e1['matchId']==e2['matchId']:
			
            if e1['matchPeriod']==e2['matchPeriod']:

                if e1['teamId']==e2['teamId']:
                    posesion.append(e1)
                
                else:
                    posesion.append(e1)

                    if len(posesion)>3:

                        team1 = e1['teamId']
                        match= e1["matchId"]

                        d1 = direct_play(posesion)
                        d2 = counterattack(posesion)
                        d3 = T_maintenance(posesion)
                        d4 = T_build_up(posesion)
                        d5 = T_zona_media(posesion)
                        d6 = flow_rate(posesion) 
                        d7 = crossing(posesion)
                        d8 = pressure_point(posesion, Tpos_ant)
                        d9 = pressure_loss(posesion,Tpos_ant)
                        d10 = shots(posesion)
                        

                        if (d1+d2+d3+d4+d5+d6+d7+d8+d9+d10) >0.1:
                            
                            Data["direct_play"].append(d1)
                            Data["counterattack"].append(d2)
                            Data["T_maintenance"].append(d3)
                            Data["T_build_up"].append(d4)
                            Data["T_zona_media"].append(d5)
                            Data["flow_rate"].append(d6)
                            Data["crossing"].append(d7)
                            Data["pressure_point"].append(d8)
                            
                            if team1!=team_ant and team1!='':
                                Data["pressure_loss"].append(d9)
                            else:
                                Data["pressure_loss"].append(0)
                            
                            Data["shots"].append(d10)

                            Data["match"].append(match)
                            Data["team1"].append(team1)
                            Data["liga"].append(liga)    

                    # ----------
                    
                    if len(posesion):
                        if e1['matchId']==e2['matchId']:
                            Tpos_ant= e2["eventSec"] - posesion[0]["eventSec"]
                            team_ant = team1
                        else:
                            Tpos_ant=0
                            team_ant =''
                    else:
                        Tpos_ant=0
                        team_ant =''
                        
                    posesion = []

            else:
                posesion = []
        
        else:
                posesion = []


df0 = pd.DataFrame.from_dict(Data)

df0["liga"]= df0['liga'].astype('category')

print (df0.shape)
df0.head()


England
France
Germany
Italy
Spain
(215681, 13)


,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,match,team1,liga
0,13.798826,0.0,0.000000,0.000000,9.790285,0.000000,0,0.0,0.000000,0,2499719,1609,England
1,0.000000,0.0,0.000000,0.000000,5.661206,0.529922,0,0.0,0.000000,0,2499719,1609,England
2,0.000000,0.0,6.901369,0.000000,0.000000,0.000000,0,24.0,0.000000,0,2499719,1631,England
3,3.621163,0.0,0.000000,7.822331,0.000000,0.511357,1,0.0,0.000000,1,2499719,1609,England
4,0.000000,0.0,0.000000,11.391057,0.000000,0.263364,1,100.0,2.566969,0,2499719,1631,England


### me quedo con los partidos que se jugaron entre los primero 6

In [29]:
l2t2r = json.load( open( "league2team2rank.json","r" ) )
l2m2ts = json.load( open( "league2match2teams.json","r" ) )

In [30]:
def f(row):

    l = row.liga
    m = str(row.match)
    t1,t2 = l2m2ts[l][m]

    r1 = l2t2r[l][t1]
    r2 = l2t2r[l][t2]

    if r1<=6 and r2<=6:
        return True
    else:
        return False
    


mask = df0.apply( lambda row:f(row), axis=1 )

In [31]:
df1 = df0[mask].reset_index(drop=True)

print (df1.shape)
df1.head()

(19267, 13)


,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,match,team1,liga
0,0.682726,0.000000,0.000000,0.000000,6.899285,0.000000,0,50.0,17.163295,0,2499737,1610,England
1,9.315857,3.161131,0.000000,0.000000,18.664204,0.000000,0,29.0,0.000000,0,2499737,1610,England
2,0.000000,0.000000,0.000000,11.450382,0.000000,0.262000,1,0.0,0.000000,0,2499737,1610,England
3,3.515553,0.000000,14.555887,0.000000,0.000000,0.000000,0,9.0,0.000000,0,2499737,1624,England
4,0.000000,0.000000,0.000000,0.000000,10.293179,0.388607,0,0.0,0.000000,0,2499737,1624,England


In [32]:
#Como las columnas siguen aproximadamente una distribución lognormal, transformo a log(1+x) para trabajar con distribuciones normales

#exclude_columns = ['liga', 'match', 'team1', "crossing", "shots"]

#cols = df0.columns.difference(exclude_columns)

#df1 = df0.copy(deep=True)

#df1[cols] = np.log1p(df1[cols])                 # transormo features a log(1+x)

#print (df1.shape)
#df1.head()

### Data agregada por partido y por equipo

In [33]:
def f(x):
    
    if x.name=="liga":
        return list(set(x))[0]

    
    elif x.name=="crossing" or x.name=="shots":  
        return np.sum(x)

    else:
        return np.sum(x)


df2 = df1.groupby(["match","team1" ]).agg( lambda x: f(x)).reset_index()#.drop("match", axis=1)


# una vex contados todos los centros y tiros del partido transformo a log
#df2["crossing"]= np.log1p(df2["crossing"])
#df2["shots"]= np.log1p(df2["shots"])



# Escaleo de las variables ------------
exclude_columns = ['liga', 'team1', 'match']                                             # Columnas a excluir del escalado

columns_to_scale = df2.columns.difference(exclude_columns)

scaler = StandardScaler()                                                               # Crear un StandardScaler

df2[columns_to_scale] = scaler.fit_transform(df2[columns_to_scale])                     # Seleccionar solo las columnas que queremos escalar

print (df2.shape)

df2.tail()

(300, 13)


,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
295,2576302,3161,0.005995,-0.659745,1.881349,-0.329227,-0.290195,-0.472221,0.022411,-0.838737,-1.117413,-0.525231,Italy
296,2576322,3158,1.697439,-0.268536,1.041298,-0.282286,0.609363,-0.315968,0.022411,0.547893,0.722411,-1.528856,Italy
297,2576322,3159,-0.189453,-1.487792,-0.347988,-0.127944,0.728053,-0.243058,-1.418274,-0.055134,0.378226,-0.525231,Italy
298,2576335,3161,0.698123,0.378928,0.624191,-0.454350,1.259016,-0.192738,-0.217703,-0.167787,-0.723864,-0.525231,Italy
299,2576335,3162,-1.175831,-0.368256,-1.298317,-0.666836,-1.746856,-0.509767,-0.457817,-1.246277,-0.648963,0.478395,Italy


In [ ]:
df2.to_csv("metrics_2_first6.csv",index=False)

### Rankings

Uso los puntos que obtuvieron en el campeonato para asignar un ranking

In [35]:
wyid2ptos= { 

    "Spain":{
        674:73,675:76,676:93,677:29,678:43,679:79,680:58,682:61,
        683:20,684:60,687:49,691:49,692:49,695:46,696:47,698:55,
        701:51,712:43,714:22,756:51
    },

    "England":{
        1609:63, 1610:70, 1611:81, 1612:75, 1613:44, 1619:36, 
        1623:49, 1624:77, 1625:100, 1627:31, 1628:44, 1631:47, 
        1633:42, 1639:33, 1644:41, 1646:54, 1651:40, 1659:44, 
        1673:37, 10531:33
    },
    "Italy":{
        3157:64, 3158:77, 3159:95, 3161:72, 3162:72, 3163:40, 3164:54, 
        3165:40, 3166:39, 3172:60, 3173:39, 3176:57, 3185:54, 3187:91, 
        3193:41, 3194:25, 3197:35, 3204:38, 3219:21, 3315:43
    },
    "France":{
        3766:78, 3767:93, 3770:52, 3771:77, 3772:55, 3774:58, 
        3775:54, 3776:38, 3777:26, 3779:38, 3780:37, 3782:55, 
        3783:38, 3785:47, 3787:51, 3789:45, 3795:33, 3799:41, 
        3804:48, 19830:80
    },
    "Germany":{
        2443:42, 2444:84, 2445:51, 2446:54, 2447:54.5, 2449:63, 
        2450:31, 2451:33, 2453:36, 2454:47, 2455:39, 2457:43, 
        2460:36, 2462:49, 2463:22, 2481:41, 2482:55, 2975:53 
    }

}

ligaEquipo2rank={ liga:{} for liga in wyid2ptos.keys() }

for liga in wyid2ptos.keys():

    rank=1

    for team, ptos in sorted(wyid2ptos[liga].items(), key=lambda a:a[1], reverse=True):

        ligaEquipo2rank[liga][team]=rank

        rank+=1


json.dump( ligaEquipo2rank, open("league2team2rank.json","w") )